In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization


from data_cleaning import clean_df
from KFPF_lambda_cuts import KFPF_lambda_cuts
from plot_tools import AMS, preds_prob, plot_confusion_matrix, plt_sig_back
import tree_importer 
import uproot


#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(7)

import gc
import ROOT

In [ ]:
df_clean_signal = uproot.open('/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/dcm/dcm_signal.root:plain_tree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
aa = ['b', 'chi2geo','chi2primneg', 'chi2primpos', 'chi2_topo', 'cosine_first', 'cosine_second', 'cosine_topo', 'distance', 'eta', 'l', 'ldl', 'mass', 'p', 'pT', 'phi', 'px', 'py', 'pz', 'rapidity', 'vtx_chi2_first', 'vtx_chi2_second',
 'z_first', 'z_second', 'z_smaller', 'M', 'issignal', 'id', 'nhits_mvd_first', 'nhits_mvd_second', 'nhits_mvd_sum', 'nhits_tot_first', 'nhits_tot_second', 'nhits_tot_sum', 'pid']
df_clean_signal.columns=aa
#signal = df_clean_signal[ (df_clean_signal['mass']>df_clean_signal['mass'].mean()-1.5*df_clean_signal['mass'].std())
#               & (df_clean_signal['mass']<df_clean_signal['mass'].mean()+1.5*df_clean_signal['mass'].std()) & (df_clean_signal['M']>200) & (df_clean_signal['M']<400)]
signal = df_clean_signal[ (df_clean_signal['mass']>df_clean_signal['mass'].mean()-1.5*df_clean_signal['mass'].std())
               & (df_clean_signal['mass']<df_clean_signal['mass'].mean()+1.5*df_clean_signal['mass'].std()) &(df_clean_signal['pT']<=0.6) & (df_clean_signal['rapidity']<=1.6) &(df_clean_signal['M']>200) &(df_clean_signal['M']<400)]
del df_clean_signal
#signal["issignal"].replace({3: 2, 4: 2, 5:2, 6:2}, inplace=True)
#signal["issignal"].replace({1: 0, 2: 1}, inplace=True)
#signal['original_issignal']=signal['issignal']
signal['issignal']=((signal['issignal']>0)*1)

In [ ]:
signal['Candidates_generation']=signal['issignal']
signal['Candidates_chi2_geo']=signal['chi2geo']
signal['Candidates_chi2_prim_first']=signal['chi2primneg']
signal['Candidates_chi2_prim_second']=signal['chi2primpos']
signal['Candidates_distance']=signal['distance']
signal['Candidates_l_over_dl']=signal['ldl']
signal['Candidates_mass']=signal['mass']


del signal['issignal']


In [ ]:
signal['Candidates_plain_generation']=signal['issignal']
signal['Candidates_plain_chi2_geo']=signal['chi2geo']
signal['Candidates_plain_chi2_prim_first']=signal['chi2primneg']
signal['Candidates_plain_chi2_prim_second']=signal['chi2primpos']
signal['Candidates_plain_distance']=signal['distance']
signal['Candidates_plain_l_over_dl']=signal['ldl']
signal['Candidates_plain_mass']=signal['mass']
signal['Candidates_plain_chi2_topo']=signal['chi2_topo']
signal['Candidates_plain_cosine_topo']=signal['cosine_topo']


del signal['issignal']


In [ ]:
del signal['chi2geo']
del signal['chi2primneg']
del signal['chi2primpos']
del signal['distance']
del signal['ldl']
del signal['mass']
del signal['chi2_topo']
del signal['cosine_topo']

In [ ]:
del df_clean_urqmd['Candidates_cosine_topo']
del df_clean_urqmd['Candidates_eta']
del df_clean_urqmd['Candidates_pT_err']
del df_clean_urqmd['Candidates_pid']
del df_clean_urqmd['Candidates_pz']
del df_clean_urqmd['Candidates_py']

In [ ]:
df_clean_urqmd.columns
gc.collect()

In [ ]:
signal.shape[0]/df_clean_urqmd[df_clean_urqmd['Candidates_generation']==0].shape[0]

In [ ]:
df_clean_dcm= uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/pfsimple/outputs/apr20_fr_18.2.1_fs_jun19p1/dcm/auau/12agev/mbias/sis100_electron_target_25_mkm/dcm_back.root:PlainTree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
df_clean_dcm['Candidates_generation']=((df_clean_dcm['Candidates_generation']>0)*1)

df_clean_urqmd = uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/scripts_treelite/urqmd_signal_dcm_back/urqmd_signal_pfsimple_output_2M.root:PlainTree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
df_clean_urqmd['Candidates_generation']=((df_clean_urqmd['Candidates_generation']>0)*1)

In [ ]:
df_clean_urqmd[df_clean_urqmd['Candidates_plain_generation']==1].shape
df_clean_dcm[df_clean_dcm['Candidates_plain_generation']==0].shape

In [ ]:
df_clean_urqmd = uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/2M_files_analysis/urqmd_signal_only_after_plain_tree_2M//urqmd_signal.root:plain_tree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)
#df_clean_urqmd.columns = a
#df_clean_urqmd = df_clean_urqmd[df_clean_urqmd["issignal"]>0]
#df_clean_urqmd["issignal"].replace({3: 2, 4: 2, 5:2, 6:2}, inplace=True)
#df_clean_urqmd["issignal"].replace({1: 0, 2: 1}, inplace=True)
#df_clean_urqmd['primary_secondary']=df_clean_urqmd['issignal'].copy()
#df_clean_urqmd['original_issignal'] = df_clean_urqmd['Candidates_generation']
df_clean_urqmd['Candidates_plain_generation']=((df_clean_urqmd['Candidates_plain_generation']>0)*1)
df_clean_urqmd= df_clean_urqmd[(df_clean_urqmd['Candidates_plain_M']>200)&(df_clean_urqmd['Candidates_plain_M']<300)]

In [ ]:
df_clean_dcm= uproot.open('/home/shahid/Mount/gsi/u/Mount/lustre/khan/scripts_treeplainer/outputs/apr20_fr_18.2.1_fs_jun19p1/dcm/auau/12agev/mbias/sis100_electron_target_25_mkm/dcm_back.root:plain_tree',decompression_executor=executor,
                                  interpretation_executor=executor).arrays(library='pd',decompression_executor=executor,
                                  interpretation_executor=executor)

df_clean_dcm= df_clean_dcm[(df_clean_dcm['Candidates_plain_M']>200)&(df_clean_dcm['Candidates_plain_M']<300)]

In [ ]:
df_clean_urqmd.shape

In [ ]:
df_clean_urqmd['Candidates_plain_generation']=((df_clean_urqmd['Candidates_plain_generation']>0)*1)

In [ ]:
np.unique(df_clean_urqmd['Candidates_plain_generation'])

In [ ]:
def signal_back(MC, data, mass_variable, generation_id,signal_to_back_ratio):
    signal_selected= MC[(MC[mass_variable]>1.1) & (MC[mass_variable]<1.13) & (MC[generation_id]==1)]
    background_selected = data[(data[generation_id] == 0)
                & ((data[mass_variable] > 1.077)
                & (data[mass_variable] < 1.1) | (data[mass_variable]>1.13) 
                   & (data[mass_variable] < 1.2))].sample(n=signal_to_back_ratio*(signal_selected.shape[0]))

    df_scaled = pd.concat([signal_selected, background_selected])
    del signal_selected, background_selected
    return df_scaled

df_scaled = signal_back( df_clean_urqmd, df_clean_dcm,'Candidates_plain_mass', 'Candidates_plain_generation', 3)

gc.collect()
cuts = [ 'Candidates_plain_chi2_geo', 'Candidates_plain_chi2_prim_first', 'Candidates_plain_chi2_prim_second', 'Candidates_plain_distance', 'Candidates_plain_l_over_dl']
#cuts = [ 'chi2geo', 'chi2primneg', 'chi2primpos', 'chi2_topo', 'cosine_topo', 'distance','ldl','l', 'nhits_mvd_first','nhits_mvd_second','nhits_tot_first','nhits_tot_second','vtx_chi2_first','vtx_chi2_second']
x = df_scaled[cuts].copy()
y =pd.DataFrame(df_scaled['Candidates_plain_generation'], dtype='int8')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=324, stratify=y)
del x, y
dtrain = xgb.DMatrix(x_train, label = y_train)
gc.collect()

In [ ]:
fig, axs = plt_sig_back(df_scaled,'Candidates_plain_generation', 'Candidates_plain_mass')
fig.text(0.14, 0.85, r'DCM-QGSM-SMM', color = 'red',  fontsize=13)
fig.text(0.14, 0.8, r'Au+Au @ 12 $A$GeV/$c$', color = 'red',  fontsize=13)
fig.text(0.14, 0.75, r'URQMD', color = 'blue', fontsize=13)
fig.text(0.14, 0.7, r'Au+Au @ 12 $A$GeV/$c$', color = 'blue', fontsize=13)
fig.savefig("hists.png")

del df_scaled

In [ ]:
def bo_tune_xgb(max_depth, gamma, alpha, n_estimators ,learning_rate,scale_pos_weight):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'alpha':alpha,
              'n_estimators': n_estimators,
              'learning_rate':learning_rate,'scale_pos_weight':scale_pos_weight,
              'subsample': 0.8,
              'eval_metric': 'auc','tree_method':'hist','objective':'binary:logistic', 'nthread' : 7}
    cv_result = xgb.cv(params=params, dtrain=dtrain, num_boost_round=10, nfold=5)
    return  cv_result['test-auc-mean'].iloc[-1]

xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (4, 10),
                                             'gamma': (0, 1),
                                            'alpha': (2,20),
                                             'learning_rate':(0.01,1),
                                             'n_estimators':(100,1000),'scale_pos_weight':(1,10)
                                            })

#performing Bayesian optimization for 5 iterations with 8 steps of random exploration with an #acquisition function of expected improvement
xgb_bo.maximize(n_iter=5, init_points=5, acq='ei')

In [ ]:
max_param = xgb_bo.max['params']
param= {'alpha': max_param['alpha'], 'gamma': max_param['gamma'], 'learning_rate': max_param['learning_rate'],
        'max_depth': int(round(max_param['max_depth'],0)), 'n_estimators': int(round(max_param['n_estimators'],0)), 
         'objective':'binary:logistic','tree_method':'hist','nthread' : 7}

#Fit/train on training data
bst = xgb.train(param, dtrain, num_boost_round=20)

#predicitions on training set
bst_train= pd.DataFrame(data=bst.predict(dtrain),  columns=["xgb_preds"])
y_train=y_train.set_index(np.arange(0,bst_train.shape[0]))
bst_train['Candidates_plain_generation']=y_train['Candidates_plain_generation']

#predictions on test set
dtest1=xgb.DMatrix(x_test, label = y_test)
bst_test = pd.DataFrame(data=bst.predict(dtest1),  columns=["xgb_preds"])
y_test=y_test.set_index(np.arange(0,bst_test.shape[0]))
bst_test['Candidates_plain_generation']=y_test['Candidates_plain_generation']

#ROC cures for the predictions on train and test sets
train_best, test_best = AMS(y_train, bst_train['xgb_preds'],y_test, bst_test['xgb_preds'])

#The first argument should be a data frame, the second a column in it, in the form 'preds'
fig, ax = preds_prob(bst_train,'xgb_preds', 'Candidates_plain_generation',bst_test,'xgb_preds', 'Candidates_plain_generation')
fig.tight_layout()
fig.savefig("hists.png")
#del x_train, x_test, y_train, y_test, bst_train, bst_test

In [ ]:
dtest2 = xgb.DMatrix(df_clean_dcm[cuts], label = pd.DataFrame(df_clean_dcm['Candidates_plain_generation'], dtype='int8'))
df_clean_dcm['xgb_preds'] = bst.predict(dtest2)
del dtest2 
gc.collect()

df_clean_dcm[df_clean_dcm['xgb_preds']>0.9]['Candidates_plain_mass'].hist(bins=100)

In [ ]:
bst.save_model('/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/pt_y_models/dcm_signal_urqmd_back_pt_0_0.6_rapdity_0_1.6.model')

In [ ]:
import treelite
#create an object out of your model, bst in our case
model = treelite.Model.from_xgboost(bst)
#use GCC compiler
toolchain = 'gcc'
#parallel_comp can be changed upto as many processors as one have
model.export_lib(toolchain=toolchain, libpath='/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/pt_y_models/urqmd_signal_dcm_back_multi_200_400_pt_0_0.6_rapdity_0_1.6.model.so',
                 params={'parallel_comp': 8}, verbose=True)

# Operating system of the target machine
platform = 'unix'
# C compiler to use to compile prediction code on the target machine
toolchain = 'gcc'
# Save the source package as a zip archive named mymodel.zip
# Later, we'll use this package to produce the library mymodel.so.
model.export_srcpkg(platform=platform, toolchain=toolchain,
                    pkgpath='/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/pt_y_models/urqmd_signal_dcm_back_multi_200_400_pt_0_0.6_rapdity_0_1.6.model.zip', libname='/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/urqmd_signal_dcm_back_multi_200_400_pt_0_0.6_rapdity_0_1.6.model.so',
                    verbose=True)

In [ ]:
x_train_small = x_test[0:10000]
y_train_small = y_test[0:10000]
dtrain_shap = xgb.DMatrix(x_train_small, label = y_train_small)

import shap
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(dtrain_shap)

shap.summary_plot(shap_values, x_train_small, plot_size=[10,8], show=False)
_, h = plt.gcf().get_size_inches()
plt.gcf().set_size_inches(h, h)
plt.savefig("hists.png")